In [ ]:
from sklearn.preprocessing import RobustScaler
import numpy as np
# importing pandas as pd
import pandas as pd
import re

# read an excel file and convert
# into a dataframe object
df = pd.DataFrame(pd.read_excel("Raw_Data_Customer_2023.xlsx"))

# show the original dataframe
df



,ID,First Name,Last Name,Gender,Age,Birthday,Items Sold,Email,Phone
0,573819,LIAM,Garcia,Male,13.0,1990-05-10,18.0,liam.g@example.com,+63 9274836192
1,942736,SOPHIA,Hernandez,Female,38.0,1985-09-15,11.0,sophia.h@example.com,+63 9359017268
2,208457,NOAH,Lopez,M,31.0,1992-03-20,16.0,noah.l@example.com,+63 9263841509
3,695813,MIA,Martinez,Female,NaN,1988-07-25,5.0,MIA.M@EXAMPLE.COM,+63 9316852740
4,314972,Ethan,GONZALEZ,Male,29.0,1995-01-30,20.0,ETHAN.G@EXAMPLE.COM,+63 9367254810
5,867205,ISABELLA,Perez,Female,30.0,1993-04-05,18.0,ISABELLA.P@EXAMPLE.COM,+63 9286574903
6,530961,JAMES,Ramirez,Male,33.0,1990-08-10,10.0,JAMES.R@example.com,+63 9328640795
7,147826,Amelia,TORRES,F,36.0,1987-11-15,NaN,AMELIA.T@EXAMPLE.COM,+63 9345967021
8,689374,ALEXANDER,Sanchez,Male,29.0,1994-06-20,13.0,ALEXANDER.S@EXAMPLE.COM,+63 9398561243
9,420583,EVELYN,Flores,Female,13.0,1991-02-25,13.0,EVELYN.F@EXAMPLE.COM,+63 9301756943


In [ ]:
# copy data frame
cleaned_df  = df

# convert to upper case
def convert_to_upper_case(column):
    cleaned_df[column] = cleaned_df[column].str.upper()

# convert diff inputs from gender to one value
cleaned_df = cleaned_df.replace({"Male": "MALE", "M": "MALE", "Female": "FEMALE", "F": "FEMALE"})

# reformat date into YYYY-MM-DD
cleaned_df["Birthday"] = pd.to_datetime(cleaned_df["Birthday"])

# Replace Email NaNs with first_name.last_name_initial@example.com
# Take First_Name + "." + Last_Name_Initial + "@example.com"

cleaned_df['Email'] = cleaned_df['Email'].fillna(cleaned_df['First Name'] + "." + cleaned_df['Last Name'].str[0] + '@example.com')

# convert email to lower case
cleaned_df['Email'] = cleaned_df['Email'].str.lower()

convert_to_upper_case('First Name')
convert_to_upper_case('Last Name')

def replace_outliers(data, columns):
  no_nan = cleaned_df.dropna()  # Drop rows with NaN values

  scaler = RobustScaler()  # Create a RobustScaler object

  # Select numeric columns for scaling (modify if needed)
  numeric_data = no_nan.select_dtypes(include=['float64', 'int64'])

  scaler.fit(numeric_data[columns])  # Fit scaler to specified columns

  # Apply scaling to selected columns
  scaled_data = scaler.transform(numeric_data[columns])

  # Define outlier thresholds (adjust based on your needs)
  outlier_thresholds = {'Items Sold': (1, -1), 'Age': (2, -2)}

  for col, (upper, lower) in outlier_thresholds.items():
      outliers = (scaled_data[:, columns.index(col)] > upper) | (scaled_data[:, columns.index(col)] < lower)
      outlier_indices = numeric_data.index[outliers]
      print(outlier_indices)

      # Calculate column-wise average (excluding outliers)
      average_value = round(np.average(no_nan[col]))

      # Replace outliers in original data with average
      cleaned_df.loc[outlier_indices, col] = average_value

      # Fill remaining missing values with average
      cleaned_df[col] = cleaned_df[col].fillna(average_value)
  return cleaned_df

cleaned_df = replace_outliers(cleaned_df.copy(), ['Items Sold', 'Age'])  # Use a copy to avoid modifying original data

def clean_phone(phone):
    if phone.startswith('+'):
        phone = phone[3:]  # Remove the leading + and number prefix
    return phone.replace('-', '').replace(' ', '')

# Apply the function to the Phone column
cleaned_df['Phone'] = cleaned_df['Phone'].apply(clean_phone)


Int64Index([4, 12, 13], dtype='int64')
Int64Index([0, 9], dtype='int64')


In [ ]:
cleaned_df

,ID,First Name,Last Name,Gender,Age,Birthday,Items Sold,Email,Phone
0,573819,LIAM,GARCIA,MALE,29.0,1990-05-10,18.0,liam.g@example.com,9274836192
1,942736,SOPHIA,HERNANDEZ,FEMALE,38.0,1985-09-15,11.0,sophia.h@example.com,9359017268
2,208457,NOAH,LOPEZ,MALE,31.0,1992-03-20,16.0,noah.l@example.com,9263841509
3,695813,MIA,MARTINEZ,FEMALE,29.0,1988-07-25,5.0,mia.m@example.com,9316852740
4,314972,ETHAN,GONZALEZ,MALE,29.0,1995-01-30,75.0,ethan.g@example.com,9367254810
5,867205,ISABELLA,PEREZ,FEMALE,30.0,1993-04-05,18.0,isabella.p@example.com,9286574903
6,530961,JAMES,RAMIREZ,MALE,33.0,1990-08-10,10.0,james.r@example.com,9328640795
7,147826,AMELIA,TORRES,FEMALE,36.0,1987-11-15,75.0,amelia.t@example.com,9345967021
8,689374,ALEXANDER,SANCHEZ,MALE,29.0,1994-06-20,13.0,alexander.s@example.com,9398561243
9,420583,EVELYN,FLORES,FEMALE,29.0,1991-02-25,13.0,evelyn.f@example.com,9301756943
